# 1. Working Hours
店面 coding 估算working hours
给定 working hours 的window（比如说 MON-FRI, 9-17 ），开始工作的date，和时间，结束工作的date和时间，求总共工作了多少小时

给你个例子
假设这个是 # working hours: MON-FRI, 9-17
那么 # start: WED 10 -> # end: WED 18 就是7小时
因为 # WED 10-17 working hours # 17-18 not working hours
假设 # start WED 23 # end THU 1 那么working hour 是0

然后问题海可以延伸到给定的不仅仅是 day of week，还可以有不同的week，来估计working hours
比如说 # week 0, WED, 10 -> # week 200, MON, 7

start_day = 2 (WED), 10, 2, 18 => 7


In [1]:
def working_hours(
    start_week, start_day, start_hour,
    end_week, end_day, end_hour,
    work_days=(0,1,2,3,4),  # MON-FRI
    work_start=9, work_end=17
):
    # 计算总小时数
    # 1. 统一起止时间为分钟数
    start_total_hours = start_week * 7 * 24 + start_day * 24 + start_hour
    end_total_hours = end_week * 7 * 24 + end_day * 24 + end_hour
    if end_total_hours <= start_total_hours:
        return 0

    total_hours = 0
    # 遍历每个小时
    for h in range(start_total_hours, end_total_hours):
        week = h // (7*24)
        day = (h // 24) % 7
        hour = h % 24
        if day in work_days and work_start <= hour < work_end:
            total_hours += 1
    return total_hours

# 示例
# week 0, WED(2), 10 -> week 0, WED(2), 18
print(working_hours(0, 2, 10, 0, 2, 18))  # 输出 7

# week 0, WED(2), 23 -> week 0, THU(3), 1
print(working_hours(0, 2, 23, 0, 3, 1))  # 输出 0

# week 0, WED(2), 10 -> week 200, MON(0), 7
print(working_hours(0, 2, 10, 200, 0, 7))    

7
0
7983


# 2. Longest words LC720


Given an array of strings, return the longest word that can be built one character at a time by other words in the array.

Input: ["s", "sn", "sna", "snap", "chat"]
Output: "snap"


In [ ]:
def longest_word(words):
    words_set = set(words)
    words.sort(key=lambda x: (-len(x), x))  # 长的优先，字典序小的优先， nlogn
    for word in words:
        # 检查 word 的每个前缀都在 words_set 里
        if all(word[:i] in words_set for i in range(1, len(word))): # time: O(nk), k is the longest word in the words
            return word
    return ""

# total O(nlogn + nk)

# 示例
print(longest_word(["s", "sn", "sna", "snap", "chat"]))  # 输出: snap

snap


In [8]:
from collections import deque
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_word = False
        self.word = ""

class Trie:
    def __init__(self):
        self.root = TrieNode()
    
    def insert(self, word):
        node = self.root 
        for c in word:
            if c not in node.children:
                node.children[c] = TrieNode()
            node = node.children[c]
        node.is_word = True
        node.word = word  # ✅ 插入时记录单词

    def search(self, word):
        node = self.root 
        for c in word:
            if c not in node.children:
                return False
            node = node.children[c]
        return node.is_word

    def startsWith(self, prefix):
        node = self.root 
        for c in prefix:
            if c not in node.children:
                return False
            node = node.children[c]
        return True

    def longest_word_with_all_prefixes(self):
        q = deque([self.root])
        res = ""

        while q:
            node = q.popleft()
            for ch in sorted(node.children.keys()):
                child = node.children[ch]
                if child.is_word:
                    q.append(child)
                    # 更新最长合法词
                    if len(child.word) > len(res) or \
                       (len(child.word) == len(res) and child.word < res):
                        res = child.word
        return res
    
words = ["go", "golang", "g", "gol", "gola"]
trie = Trie()
for w in words:
    trie.insert(w)

print(trie.longest_word_with_all_prefixes())  # 输出: "gola"

gola


# 3. Meeting Rooms

findmeetingroom(A, B, room)
给定A location ， B  location和 room的locations，找到两者之间开会距离最短的room


In [3]:
def findMeetingRoom(A, B, rooms):
    def dist(p1, p2):
        return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])  # 曼哈顿距离

    min_dist = float('inf')
    best_room = None

    for room in rooms:
        d = (dist(room, A) + dist(room, B)) / 2
        if d < min_dist:
            min_dist = d
            best_room = room

    return best_room


# 4. Tell if if's Permutation
Coding 是 判断一串数字是不是 permutation，permutation 的定义是 length 是n + 1 的 list里面所有数字在 [0, n] 并且没有duplicate，一开始比较简单就是O(n) 然后follow up 说能不能 O(1) memory

问了两个followup： 1） 如果不是严格increasing, 可以equal，怎么改； 2）要求打出path
两个followup都要求在原代码基础上改写，能跑


In [ ]:
def is_permutation(nums):
    # O(n) space
    n = len(nums)-1
    seen = set()
    for x in nums:
        if x < 0 or x > n or x in seen:
            return False
        seen.add(x)
    return True

def is_permutation_space_optimal(nums):
    n = len(nums)-1
    for i in range(n+1):
        # inplace swap，将nums[i]  放在正确的位置，val应该当在 nums[val]上面，但是如果发生冲突了，说明有重复或者不合法的元素
        if nums[i] < 0 or nums[i] > n or nums[nums[i]] == nums[i]:
            return False
        nums[nums[i]], nums[i] = nums[i], nums[nums[i]]
    return True

In [ ]:
# if nums can be euqal, not strictly increasing
# 那么不需要判断重复即可
def is_permutation_with_equal(nums):
    n = len(nums) - 1
    for x in nums:
        if x < 0 or x > n:
            return False
    return True

# 要求打印出path（置换环）
def print_permutation_cycles(nums):
    n = len(nums)
    visited = [False] * n
    for i in range(n):
        if not visited[i]:
            path = []
            j = i
            while not visited[j]:
                path.append(j)
                visited[j] = True
                j = nums[j]
            print(path)

# 5. Number of islands variants
Euclident distance cluster (variation of LC "number of island"  ): find_clusters


In [9]:
def find_clusters(points, D):
    """
    :type points: List[p1=(x1,y1), p2, p...]
    :rtype: int

    time complexity: O(n^2)
    space complexity: O(n), N is nuber of poins

    visited list
    go throught all the neightbors and update the current cluster

    dfs only updated visited and clustering one at the begining
    """

    def euclidean_distance(P1, P2):
        x1, y1 = P1
        x2, y2 = P2
        return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5


    def dfs(point_index=0, cluster = []):
      """
      Start from the center (0,0), loop through all the neightbors
      """
      #Add the visted point and the visted point cluster
      visited[point_index] = True
      cluster.append(points[point_index])
      for j , neightbor in enumerate(points):
        if not visited[j]: # neightbor
          if euclidean_distance(points[point_index], neightbor) <= D:
            # check on the next neightbor point
            dfs(j, cluster) #


    clusters = [] #results

    visited =  [False]* len(points) # indicate if the point as visited or not
    for i , point in enumerate(points):
        if not visited[i]:
            cluster = [] # start a new cluster
            # loop through all the neightbors and  update the element in the current cluster
            dfs(i, cluster)
            # Updated the results
            clusters.append(cluster)

    return clusters

# Example Usage
points = [(0, 0), (1, 1), (10, 10), (10, 11), (11, 10)]
D = 2.0
clusters = find_clusters(points, D)
print("Clusters:", clusters)

Clusters: [[(0, 0), (1, 1)], [(10, 10), (10, 11), (11, 10)]]


# 6. Decode ways variant

coding问了道不知道lc上有没有的，难度大概medium 91
把数字decode成字母，比如A->1， B->2...，Z->26
给定一串数字比如11106，返回所有可以decode的方法数，11106有11，10，6和1，1，10，6两种解法，因为06是invalid
给了个dfs的解，然后伪代码写了一下用dp怎么做。dp好像有点小bug但老哥没太在意


In [12]:
def num_decodings(s):
    n = len(s)
    dp = [0] * (n+1)
    dp[0] = 1
    for i in range(1, n+1):
        if s[i-1] != '0':
            dp[i] += dp[i-1]
        if i >= 2 and s[i-2] != '0' and 10 <= int(s[i-2:i]) <= 26:
            dp[i] += dp[i-2]
    return dp[n]

print(num_decodings("11106"))  # 输出: 2

2


# 7. Find local minimum
Coding:问怎么寻找局部极小值，回答通过二分查找，以及follow up怎么在存在极小值的数组里面找某个target值的最小index.回答先找到极小值，然后先后在极小值左边和右边二分查找target值. like 162?


# 8. Min cost for getting all fines

coding没有见过不是Leetcode原题。因为是9月多面的，具体题目记不清了。但是大概是，有一个文件被存在很多的machine上，然后每个machine有一些bits， 会有起始和结束index。和每个partial 文件的cost。这些partial文件会有overlap。问最后能得到全部文件的最小cost。没有做出来。我感觉可能是用DP.for k in range(i, j):​    dp[i][j] = min(dp[i][k]+dp[k+1][j], dp[i][j])


In [13]:
def min_cost_cover(N, intervals):
    # intervals: [(start, end, cost), ...]
    dp = [[float('inf')] * N for _ in range(N)]
    for start, end, cost in intervals:
        dp[start][end] = min(dp[start][end], cost)
    for length in range(1, N+1):
        for i in range(N-length+1):
            j = i+length-1
            for k in range(i, j):
                dp[i][j] = min(dp[i][j], dp[i][k]+dp[k+1][j])
    return dp[0][N-1] if dp[0][N-1] != float('inf') else -1

# 9. Online Counter (Circular buffer) LC362

设计一个online counter class，有两个method，一个get，一个increment，每次increment被call的时候相当于一个线上请求，每次get的时候返回过去5分钟的请求数，要求time和space都得是O1， 要写一下mock test。

In [4]:
class OnlineCounter:
    def __init__(self):
        self.buckets = [[0, 0] for _ in range(300)]  # 每个 [timestamp, count]

    def increment(self, t: int):
        idx = t % 300
        if self.buckets[idx][0] != t:
            self.buckets[idx] = [t, 1]
        else:
            self.buckets[idx][1] += 1

    def get(self, t: int) -> int:
        total = 0
        for ts, count in self.buckets:
            if t - ts < 300:
                total += count
        return total

# 


56
127
166
39
32
329
827
coding： 二分题 简单的 就是给时间戳让你找出/count在这个点之前的所有event


In [ ]:
def flat_array(arr):
    res = []
    for item in arr:
        if isinstance(item, list):
            res.extend(flat_array(item))
        else:
            res.append(item)
    return res

flat_array([1, [2, 3, [4, 5], 6], 7])


[1, 2, 3, 4, 5, 6, 7]